# First Look at Imaging
*Modular CASA and Interactive Clean*

This guide is written for CASA x.x.x.x and uses Python 3.

The purpose of this tutorial is to provide a first look at imaging ALMA data for those new to CASA.

Data delivered by ALMA is pre-calibrated either by ARC staff or by the ALMA calibration pipeline. The delivered data is ready for imaging. This tutorial demonstrates the basic procedures that will help you complete the imaging steps.

This guide covers the same material used in hands-on training sessions at [NRAO Community Days events](https://science.nrao.edu/facilities/alma/naasc-workshops/facilities/alma/community) and ALMA Data Reduction tutorials presented by NAASC staff.

Previous versions of "Imaging Tutorials for CASA Beginners" guides can be found [here](https://casaguides.nrao.edu/index.php?title=Archived_ALMA_CASAguides). The task tclean is only available in CASA 4.7 and later, and interactive clean is only available in CASA X.X and later.

## Getting CASA

If you do not already have the needed CASA Python packages installed on your machine, you will have to download and install it. We recommend using a Python `venv` for this tutorial and have supplied a `requirements.txt` for easier setup.

Download and installation instructions are available [here](http://casa.nrao.edu/casa_obtaining.shtml).

For this tutorial, with modular CASA, installation is as easy as a few pip installs. We've provided a 'requirements.txt' to work from that should install every necessary dependency.

### Create a virtual environment with Python3.8
```python3.8 -m venv first_look_venv```

### Activate the virtual environment
```source first_look_venv/bin/activate```

### Download necessary packages
```pip3 install -r requirements.txt```


## Import the Needed Packages

In [ ]:
import os
import sys

from bokeh.plotting import output_notebook
from bokeh.io import show
from bokeh.models import Title
from bokeh.layouts import row, column

# TODO: Remove when casagui#mh/jupyter branch is merged
# Locate custom casagui directory with Jupyter capability
custom_casagui_path = ''
sys.path.insert(0, os.path.abspath(custom_casagui_path))

import casatasks
from casagui.apps import InteractiveClean, MakeMask
output_notebook()

### Display Helper function

In [ ]:
def disp(image_path, title):
    im = MakeMask(image_path)
    im._cube.image().add_layout(Title(text=title, align="center"), "above")
    return im._cube.image()

## About the Sample Data: Continuum and N2H+ in TW Hydra

The data for this example comes from ALMA Project 2011.0.00340.S, "Searching for H2D+ in the disk of TW Hya v1.5", for which the PI is Chunhua Qi. Part of the data for this project has been published in [Qi et al. 2013](http://adsabs.harvard.edu/abs/2013Sci...341..630Q).

The original observation had three scientific objectives:

1. Image the submm continuum structure in TW Hydra
2. Image the H2D+ line structure (rest frequency 372.42138 GHz)
3. Image the N2H+ line structure (rest frequency 372.67249 GHz)

The data used in our tutorial has already been calibrated. Furthermore, to make the data set more manageable, we have reduced it in size by averaging in time and frequency. Our goal will be to image the continuum emission and the N2H+ spectral line, which is bright and well suited for demonstrating the imaging techniques.

The spectral window we will image covers 234.375 MHz in bandwidth, and contains 384 channels spaced by 610 kHz. The data includes observations from 21 of the ALMA 12-m main array antennas, observed during Early Science Cycle 0.



## Getting the Data

The data used in this tutorial is part of a larger data package used for NRAO calibration and imaging tutorials. If you plan on working through all of the "First Look" tutorials for CASA beginners, it is worth downloading the entire package now. If you plan on working through just this imaging tutorial then you can download a smaller data package (see below).

The complete package (4.1G) is available here:

https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya.tgz

You can then unpack the data as follows:

In [ ]:
!rm -rf MyTutorial
!mkdir MyTutorial
%cd MyTutorial

In [ ]:
!tar -xvzf sis14_twhya.tar.gz

Alternatively, if you do not want to download the full 4.1G data package, you can download just the smaller data file (600M) needed specifically for this tutorial as follows:

In [ ]:
!wget -r -np -nH --cut-dirs=4 --reject "index.html*" https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya_calibrated_flagged.ms.tar
!tar -xf sis14_twhya_calibrated_flagged.ms.tar

Or you can download the file via your browser at the following links: https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya_calibrated_flagged.ms.tar

## Starting CASA

At this point you should be working in a unix shell from the directory that contains the data. An "ls" command should show you the data file "sis14_twhya_calibrated_flagged.ms" in the current working directory.

In [ ]:
pwd

In [ ]:
!ls

Rather than using monolithic CASA like the previous tutorials, we will use a selection of its components and tools. 

## Getting Oriented with the Data

Let's examine the header information and data structure of our downloaded data set using [listobs](https://casa.nrao.edu/casadocs/latest/global-task-list/task_listobs/about). In the modular version of CASA, this is found under the `casatasks` package.

In [ ]:
casatasks.listobs(vis='sis14_twhya_calibrated_flagged.ms')

## Inspecting the Data

There are various ways we can inspect the data before cleaning it.

We can plot the antenna array layout for this observation with the `plotants` CASA task.

In [ ]:
casatasks.plotants(vis='sis14_twhya_calibrated_flagged.ms/')

We can also view some of the statistics of the measurement set directly with `visstat`.

In [ ]:
casatasks.visstat(vis='sis14_twhya_calibrated_flagged.ms/')

## First Look at TCLEAN

Before we use interactive clean, let's first look tclean (which interactive clean provides an interactive user interface to).

Starting with CASA 4.7, the imaging capabilities of [clean](https://casa.nrao.edu/casadocs/latest/global-task-list/task_clean/about) have been refactored and improved in a task called [tclean](https://casa.nrao.edu/casadocs/latest/global-task-list/task_tclean/about), which is called by the user in largely the same way as clean. In the guide [TCLEAN and ALMA](https://casaguides.nrao.edu/index.php?title=TCLEAN_and_ALMA), the differences in syntax between clean and tclean are made more explicit. For reference, legacy first look guides for older versions of CASA that use clean rather than tclean or interactive clean for imaging are kept in [Archived ALMA CASAguides](https://casaguides.nrao.edu/index.php?title=Archived_ALMA_CASAguides).

In this dataset, the phase calibrator is identified by field 3. Let's image this calibrator into an image file called "phase_cal". First, in case you've run this task before, let's remove old versions of the image that use this name. (The ".*" is needed because imaging produces several files with the same root name).

In [ ]:
!rm -rf phase_cal.*

Now we will use the task tclean to do the imaging. Here is an explanation of how we set some of the parameters; see the documentation for a complete list of tclean options.

- We will first image the phase calibrator, and set the image name to "phase_cal". To identify the phase calibrator in a measurement set, look at the 'ScanIntent' column in the output of listobs. The phase calibrator should have the intent "CALIBRATE_PHASE#ON_SOURCE". In this MS, the phase calibrator is J1037-295, identified by field = "3".

- We will use multifrequency synthesis (specmode = mfs) to make a single continuum image. Multifrequency synthesis combines data from all selected spectral channels into a single continuum image. If your observations cover a large frequency range, it is possible that the amplitude or structure of the source can change substantially with frequency. This is only a concern if the fractional bandwidth (delta_nu/nu_center) is greater than 10%. For this dataset it is not an issue. Therefore, we set deconvolver=hogbom, which uses nterms=1, telling tclean that each deconvolved component has a single amplitude at all frequencies.

- We will image a single pointing, so we set gridder='standard'. If we had multiple, contiguous pointings-- or if we were simultaneously imaging 7m and 12m array data (even if only a single pointing of each)--- we would need to set gridder='mosaic'.

- We set the cell size to 0.1 arcseconds, which places ~5 pixels across a beam. As a rule of thumb, we'd like ~5 pixels across the smallest direction of our elliptical beam. For a small dataset, you may be able to determine a good cell size with a quick imaging call, and note the beam calculated by tclean. For many ALMA projects, however, this could take a substantial amount of time! You can estimate what the cell size should be by looking again at the uv-coverage. Change the x-axis to uvwave, and the cell size is roughly 206265/(longest baseline in wavelengths)/(number of cells across the beam). For this MS, this works out to 0.09", which we round up to 0.1". As you will see, tclean reports a synthesized beam size of 0.58" x 0.51", making this choice just right.


- We set the image size to 128x128 (but note that factors of 2 are not magic for CASA). This is enough to cover most of the primary beam at the observing frequency, but we might want to image a wider field for a non-point source. Image sizes can be arbitrary but should be symmetrical. Good practice is probably to round up to the nearest 10 or 100 pixels and if tclean does not like your choice, it will recommend in the logger window a better choice.

Here, the tclean task will *NOT* run in interactive mode. For users familiar with clean, note that in contrast with the clean task, leaving niter (the maximum number of iterations) unset defaults to niter=0, and tclean is being told not to do any cleaning. If there are no clean components in the model -- for instance if this is the first invocation of tclean on this field and spectral window for the given set of visibilities (MS) -- then what will be created is the dirty cube or image. Here, we first run tclean with niter unset.

`tclean` is again found under the `casatasks` package.

In [ ]:
casatasks.tclean(vis='sis14_twhya_calibrated_flagged.ms',
       imagename='phase_cal',
       field='3',
       spw='',
       specmode='mfs',
       deconvolver='hogbom',
       gridder='standard',
       imsize=[128,128],
       cell=['0.1arcsec'],
       weighting='briggs',
       threshold='0.0mJy')

Let's look at the generated dirty image:

In [ ]:
im_dirty = disp('phase_cal.image', "Dirty Image")
show(im_dirty)

## Actual TCLEAN Run

Next, we'll actually run tclean on the image in non-interactive mode. This non-interactive mode can save you a lot of time and has the advantage of being very reproducible. Note, however, that best practice for an image with uncertain calibration and especially one with a bright source, is to clean interactively at least the first time. In the case where an image may be "dynamic range limited" (i.e., the quality is set by the accuracy of calibration and deconvolution) it can be hard to predict the correct threshold.

For non-interactive tclean, the three main parameters to specify are the threshold at which to stop (when the maximum residual in the tclean region is lower than this threshold, tclean stops), the mask (the region in which tclean is willing to identify signal), and the maximum number of iterations. The max iterations is not strictly required and it is generally recommended that it be used as more of a failsafe. That is, set it to a number so high that if tclean gets there something has gone wrong.

First let's take a stab at determining a clean mask. Look at the image you just made. All of the obvious emission is contained in a box that is bounded by pixel numbers in a range of something like (60,60) to (70,70). We'll set that box to be a mask using the "mask" parameter in the call to tclean. You could also set it by supplying a file (for example, one created from an earlier interactive version of tclean).

Now let's determine a stopping threshold for tclean. A clean threshold several times the rms noise is usually recommended to avoid adding false sources to the deconvolved image. For this tutorial, let's set the threshold to 7mJy (a number determined by doubling the RMS noise of an off-source region of the image). That is, you do not want tclean to treat a random noise spike as a source and deconvolve it from the image. This can be particularly problematic if you are doing self-calibration (a later lesson). Finally set niter=10000, which is a lot of iterations. We expect tclean to terminate before reaching this. For our purposes this is just a big number that's designed to keep tclean from running forever.

Run the command again, this time with niter set to some (large) number:

In [ ]:
!rm -rf phase_cal.*

In [ ]:
casatasks.tclean(vis='sis14_twhya_calibrated_flagged.ms',
       imagename='phase_cal',
       field='3',
       spw='',
       specmode='mfs',
       gridder='standard',
       deconvolver='hogbom',
       imsize=[128,128],
       cell=['0.1arcsec'],
       mask='box [ [ 60pix , 60pix] , [70pix, 70pix ] ]',
       weighting='briggs',
       threshold='0mJy',
       niter=10000)

In [ ]:
im_simple_clean = disp('phase_cal.image', "Simple Clean Image")
im_simple_clean_mask = disp('phase_cal.mask', "Simple Clean Mask")
show(row(im_simple_clean, im_simple_clean_mask))

Compared to the dirty image, we're already seeing some good improvement!

In [ ]:
show(row(im_dirty, im_simple_clean))

In [ ]:
!ls

All the resulting files have the "phase_cal" prefix, with different extensions:

.image is the final cleaned image
.mask shows the clean mask, or the area that tclean cleaned
.model is the set of modeled clean components used by tclean (in Jy/pixel)
.pb shows the primary beam response
.residual shows what was left after you stopped tclean (the "dirty" part of the final image)
.psf shows the synthesized beam
.sumwt is a single pixel image containing sum of weights per plane

So much good stuff. You can look at all of these using the CASA viewer. From within CASA, the viewer can be started with "viewer()", or "imview()". You can also start the viewer as a stand-alone unix utility using the unix command "casaviewer". Here, let's examine the tcleaned image:

## Experiment with TCLEAN

tclean includes a lot of options. Now is a good time to get a feel for what these options can do. One option that is very commonly tweaked by the user is the weighting scheme used to grid the UV data into a fourier-plane image. This weighting was "natural" in the first example (by default). Try changing it to "briggs" here and try a few different values of the robust parameter. 

Remove old versions of the image in case you have run this before:

In [ ]:
!rm -rf phase_cal_robust.*

Call tclean with briggs weighting and robust = -1. Make a clean mask and run a few cycles of tclean, until you are happy with the level of the residuals:

In [ ]:
casatasks.tclean(vis='sis14_twhya_calibrated_flagged.ms',
       imagename='phase_cal_robust',
       field='3',
       spw='',
       specmode='mfs',
       gridder='standard',
       deconvolver='hogbom',
       imsize=[128,128],
       cell=['0.1arcsec'],
       weighting='briggs',
       robust=-1.0,
       mask='box [ [ 60pix , 60pix] , [70pix, 70pix ] ]',
       threshold='0mJy',
       niter=5000)

In [ ]:
im_robust = disp('phase_cal_robust.image', "Robust Image")
show(im_robust)

Now is a good time to experiment a bit with tclean - try imaging the other calibrators (fields 0 and 2; check the 'ScanIntent' again in the listobs output) and making the image size and cell size larger and smaller.

For example, let's look at the marginally-resolved primary (flux) calibrator, Ceres (field 2; 'ScanIntent' = 'CALIBRATE_AMPLI#ON_SOURCE'):

In [ ]:
!rm -rf amp_cal_robust.*

In [ ]:
casatasks.tclean(vis='sis14_twhya_calibrated_flagged.ms',
       imagename='amp_cal_robust',
       field='2',
       spw='',
       specmode='mfs',
       gridder='standard',
       deconvolver='hogbom',
       imsize=[128,128],
       cell=['0.1arcsec'],
       weighting='briggs',
       mask='box [ [ 60pix , 60pix] , [70pix, 70pix ] ]',
       threshold='0mJy',
       niter=5000)

In [ ]:
im_robust_cal = disp('amp_cal_robust.image/', 'Robust Calibrator Image')
show(im_robust_cal)

Notice that Ceres is somewhat resolved.

If you try a really big pixel size you will see things break. It is recommended to have the pixel size small compared to the synthesized beam for tclean purposes (tclean quantizes the deconvolution in units of pixels). When the pixel size is big compared to the synthesized beam the imaging in general will degrade, even independent of how you tclean.

Note that we have to adjust our pixel mask for the new pixel size.

In [ ]:
!rm -rf amp_cal_bigpix.*

In [ ]:
casatasks.tclean(vis='sis14_twhya_calibrated_flagged.ms',
       imagename='amp_cal_bigpix',
       field='2',
       spw='',
       specmode='mfs', 
       gridder='standard',
       deconvolver='hogbom',
       imsize=[32,32],
       cell=['0.5arcsec'],
       weighting='briggs',
       mask='box [ [ 15pix , 15pix] , [18pix, 18pix ] ]',
       threshold='0mJy',
       niter=5000)

In [ ]:
im_big_pixel = disp('amp_cal_bigpix.image/', 'Big Pixel Image')
show(im_big_pixel)

To see the issues clearly here, compare the beam in this image to the one in the first image we made (with 5x smaller pixels).

In [ ]:
show(row(im_big_pixel, im_robust_cal))

## ASIDE: See the effects of calibration and flagging

This section gives an aside intended to demonstrate the effect of calibration. If you are focused on learning only imaging, feel free to skip this section.

The data that we are imaging went through a careful and detailed calibration process. What effect did this actually have? Let's image the secondary calibrator with and without calibration and with and without flagging just to get an idea of how our processing changed the final image.

First you need to copy the uncalibrated data from the working directory. This step will differ depending on where you are storing the data. The net result we want is to have a copy of the uncalibrated data in the current working directory.

If you downloaded the full data package, copy the uncalibrated data to your current working directory.

In [ ]:
!tar -xvf ../working/sis14_twhya_uncalibrated.ms.tar
!rm -rf sis14_twhya_uncalibrated.ms
!cp -r ../working/sis14_twhya_uncalibrated.ms .

If you downloaded only the calibrated data set, you can use "wget" again to download the uncalibrated data at this time:

In [ ]:
!wget -r -np -nH --cut-dirs=4 --reject "index.html*" https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya_uncalibrated.ms.tar        
!tar -xvf sis14_twhya_uncalibrated.ms.tar

Or you can download the file via your browser at the following links: https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya_uncalibrated.ms.tar

Now let's TCLEAN the uncalibrated data, again focusing on the secondary calibrator (field 3) and using the same calls as before.

In [ ]:
!rm -rf phase_cal_uncalibrated.*

In [ ]:
casatasks.tclean(vis='sis14_twhya_uncalibrated.ms',
       imagename='phase_cal_uncalibrated',
       field='3',
       spw='',
       specmode='mfs',
       gridder='standard',
       deconvolver='hogbom',
       imsize=[128,128],
       cell=['0.1arcsec'],
       weighting='natural',
       mask='box [ [ 60pix , 60pix] , [70pix, 70pix ] ]',
       threshold='0mJy',
       niter=5000)

If you can find a source to tclean then more power to you, but this is a mess. It's a good thing that we calibrated... In the raw (but still Tsys and WVR corrected) data you can see echos of the calibrator throughout the field, but the calibration is required to make the image coherent. Inspect the imaged uncalibrated data using the CASA viewer:

In [ ]:
im_uncal = disp('phase_cal_uncalibrated.image', 'Uncalibrated Image')
show(im_uncal)

Now let's see the effect that flagging had on the data. Copy the unflagged data from the working directory to our local directory:

In [ ]:
!tar -xvf ../working/sis14_twhya_calibrated.ms.tar
!rm -rf sis14_twhya_calibrated.ms")
!cp -r ../working/sis14_twhya_calibrated.ms .

or again, using wget:

In [ ]:
!wget -r -np -nH --cut-dirs=4 --reject "index.html*" https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya_calibrated.ms.tar
!tar -xvf sis14_twhya_calibrated.ms.tar

Or you can download the file via your browser at the following links: https://bulk.cv.nrao.edu/almadata/public/working/sis14_twhya_calibrated.ms.tar

Now image the calibrated but unflagged data for the secondary calibrator using the same parameters as before.

In [ ]:
!rm -rf phase_cal_unflagged.*

In [ ]:
casatasks.tclean(vis='sis14_twhya_calibrated.ms',
       imagename='phase_cal_unflagged',
       field='3',
       spw='',
       specmode='mfs',
       gridder='standard',
       deconvolver='hogbom',
       imsize=[128,128],
       cell=['0.1arcsec'],
       weighting='natural',
       mask='box [ [ 60pix , 60pix] , [70pix, 70pix ] ]',
       threshold='0mJy',
       niter=5000)

In [ ]:
im_unflagged = disp("phase_cal_unflagged.image", 'Unflagged Image')
show(im_unflagged)

In contrast to the uncalibrated data, the unflagged data are coherent, but they have clear artifacts in the residuals. Flagging has definitely improved the quality of the data, but in overall good quality data like we have here, we can see the target source.

In [ ]:
show(column(row(im_uncal, im_unflagged), row(im_dirty, im_simple_clean)))

## Image the Science Target

Of course, the whole point of calibration is to calibrate the science data. Here, the TW Hydra observations use field 5, which we know from the output of listobs, above. As the final step in the basic imaging tutorial, let's now image the continuum in TW Hydra. First, we will "split out" the science data into its own data set using the CASA task split. While not strictly necessary, this is a common step that makes managing the data easier. At the same time we will smooth (i.e. average) the data in frequency using width=8 to do the averaging. This reduces the data volume without losing much information, since we are really only interested in continuum imaging at this point. Although this data set was already designed to be manageable, this smoothing trick is good to keep in mind because ALMA can produce very large data sets.

In [ ]:
!rm -rf twhya_smoothed.ms

In [ ]:
casatasks.split(vis='sis14_twhya_calibrated_flagged.ms', field='5', width='8', outputvis='twhya_smoothed.ms', datacolumn='data')

In [ ]:
casatasks.listobs('twhya_smoothed.ms')

Now make a continuum image of the split-out data. Notice that TW Hydra has now been re-labeled as field 0 in the new data set because we split out only that field. Again we will use the multifrequency synthesis mode ("mfs") and we will use both a somewhat smaller pixel size and a somewhat bigger image size than above (because TW Hydra is extended and the beam will be somewhat smaller due to our use of "briggs" weighting). Let's set a static mask again and use a threshold roughly double the RMS noise value.

In [ ]:
!rm -rf twhya_cont.*

In [ ]:
casatasks.visstat(vis='twhya_smoothed.ms/')

In [ ]:
casatasks.tclean(vis='twhya_smoothed.ms',
       imagename='twhya_cont',
       field='0',
       spw='',
       specmode='mfs',
       gridder='standard',
       deconvolver='hogbom',
       imsize=[250,250],
       cell=['0.08arcsec'],
       weighting='briggs',
       robust=0.5,
       mask='box [ [ 110pix , 110pix] , [135pix, 135pix ] ]',
       threshold='8mJy',
       niter=5000)

Have a look at the image - TW Hydra is very bright. The residuals aren't perfect, but we will improve them in subsequent lessons.

In [ ]:
im_cont = disp("twhya_cont.image", "Continuum Image")
show(im_cont)

But wait! you might be asking. Didn't you say that there are spectral lines in this spectral window? Yes, the spectral window that we have imaged here as continuum also contains a N2H+ emission line. In this case, the N2H+ emission is faint enough that neglecting to flag the line channels before imaging makes no difference to the final continuum image. For this and the other continuum first look tutorials, we have thus ignored the line to focus on the basic steps of imaging and self-calibration. In general, however, you should flag channels containing emission lines in your own data prior to imaging the continuum. To see how to flag line emission prior to imaging the continuum in a spectral window, please see the more advanced tutorial at IRAS16293_Band9_-_Imaging.

# Interactive Clean

Let's try all of the above, but now interactively! Interactive Clean has web and native app interfaces as well, but by relying on plotting and UI packages that harness the portability of web deployment, Interactive Clean can run in Jupyter Notebooks like this!

## Initialize Interactive Clean

First, an instance of Interactive Clean needs to be created. As you can see, many of the input parameters are similar to those of 'tclean'. That's because many of them are passed onto 'tclean' directly during iterative cleaning.

In [ ]:
# Output image file name
ic_img = 'ic'
# Remove old outputs
os.system('rm -rf {ic_img}.* *.html *.log'.format(ic_img=ic_img))
# Grab data
!wget "https://casa.nrao.edu/download/devel/casavis/data/refim_point_withline-ms.tar.gz"

In [ ]:
!tar -xvf refim_point_withline-ms.tar.gz

Draw a clean mask around the visible emission using the mask tools and then run tclean until the emission from the TW Hydra disk is less than or comparable to the residuals around it. In this interactive mode of cleaning, you decide when tclean should stop, and at that point you hit the red 'Stop' button. If instead you did want to clean automatically down to a threshold, you can specify a threshold that is a small multiple of the rms noise either in the call to tclean above or by typing it in to the viewer window.

In [ ]:
ic = InteractiveClean( vis='refim_point_withline.ms/', imagename=ic_img,
                       imsize=512,
                       cell='12.0arcsec',
                       specmode='cube',
                       interpolation='nearest',
                       nchan=5,
                       start='1.0GHz',
                       width='0.2GHz',
                       pblimit=-1e-05,
                       deconvolver='hogbom',
                       threshold='0.001Jy',
                       niter=50,
                       cycleniter=10,
                       cyclefactor=3,
                       scales=[0,3,10] )

## Start Interactive Clean

To start Interactive Clean, the object created in the last cell is called. To establish the necessary websocket connections when running Interactive Clean, it'll prompt you with an ssh command that handles the port forwarding between your local computer and the remote server. If you're running IC locally, ignore this step and immediately hit `Enter`. Once you've properly established port forwarding (if necessary), hit the 'Enter' or 'Return' key when the input box to the right of the message is in focus (this should automatically be the case).

This will present you with the Interactive Clean GUI in the output of the cell. If the GUI is cutoff, disable cell output scrolling (*Shift+'O'* or Cell -> Current Outputs -> Toggle Scrolling) to expand the output window.

The Interactive Clean GUI is explained [here](https://casagui.readthedocs.io/en/latest/applications/interactive_clean.html#implementation). The user interface is virtually the same in a Jupyter Notebook as it is for a native webpage. The only quirk that Jupyter Notebooks introduce is that the keyboard shortcuts can be intercepted before being read by the Interactive Clean process. Before using a keyboard shortcut, make sure that the interactive clean cell output is `in focus`.

### IC GUI Basics

After running the below cell, you can see that there are a number of GUI elements. Most notably, on the right is the image of the current data cube channel. It's a [Bokeh](https://bokeh.org/) plot, so you can move the plot up/down/left/right or zoom in and out. To subtract the region of interest from the image mask (which defaults to mask the whole image), select the lasso tool, and draw the mask directly on the image then hit `Ctrl + S`.




---- 
After running the tclean task, you will be presented with the GUI as shown in Figure 2. In the TCLEAN viewer, make sure that your buttons are set to add a new oval mask region. You may need to click on the icon showing the "R" in an oval. Remember that the secondary calibrator has been selected because it is a point source, so at this point you should see a point source in the middle of the field. Draw an oval mask around the emission region (just the central dot). Double click inside the oval and watch it turn white (See Figure 2). When setting the clean mask, you should aim to capture the real emission and not much else. 

In the "Next Action" section of the GUI, you will notice several control buttons. Hit the green circle button to begin the cleaning process. This will run a major cycle of cleaning and then return. After the first round of cleaning, the plot displays the residual emission after the major cycle. Compare the brightness of the residuals with that of the source. When you are satisfied (or when tclean has met the residual threshold, 0 mJy by default - meaning that it stops at the first negative), click the red "X" and tclean will terminate. In this example, two rounds of cleaning work well. For more complex targets you may need many rounds of cleaning, and it is possible to update and add new tclean regions after each major cycle, based on the look of the residuals.

Have a quick look at the files that tclean has created.

In [ ]:
ic()

### Be sure to hit 'STOP' before running the next cell.
This restores the final image for subsequent viewing.

In [ ]:
ic.result()

In [ ]:
im_interactive = disp(ic_img+'.image', 'Interactive Clean Image')
show(im_interactive)

The history for the IC session can be viewed by running the following command.

In [ ]:
ic.history()

## Primary Beam Correction

An important subtlety of tclean (and clean) is that by default the image produced by tclean is not corrected for the primary beam (the field of view) of the individual dishes in the array. The primary beam response is typically a Gaussian with value 1 at the center of the field. To form an astronomically correct image of the sky, the output of tclean needs to be divided by this primary beam (or, in the case of mosaics, the combination of primary beam patterns used to make the mosaic). For tclean, there are two ways to do this.

First, you can set the parameter pbcor = True when running tclean. This will produce an additional image with the extension .image.pbcor, which is the cleaned image corrected for the primary beam.

Second, CASA stores the primary beam information needed to make this correction in the file with the .pb extension. The CASA task impbcor can be used to combine the .pb image with the output image from TCLEAN to produce a primary-beam corrected image.

First remove the old primary beam corrected image if it exists

In [ ]:
!rm -rf twhya_cont_auto.*

In [ ]:
casatasks.tclean(vis='twhya_smoothed.ms',
       imagename='twhya_cont_auto',
       field='0',
       spw='',
       specmode='mfs',
       gridder='standard',
       deconvolver='hogbom',
       imsize=[250,250],
       cell=['0.08arcsec'],
       mask='box [ [ 100pix , 100pix] , [150pix, 150pix ] ]',
       weighting='briggs',
       robust=0.5,
       threshold='15mJy',
       niter=10000)

Let's see what the image looks like before primary beam correction:

In [ ]:
im_cont_auto = disp('twhya_cont_auto.image', "Continuity Auto Image")
show(im_cont_auto)

In [ ]:
!rm -rf twhya_cont_auto.pbcor.image

Now correct the image:

In [ ]:
casatasks.impbcor(imagename='twhya_cont_auto.image',
        pbimage='twhya_cont_auto.pb',
        outfile='twhya_cont_auto.pbcor.image',
        mode="multiply", cutoff=-1)

And then compare the primary beam, uncorrected image, and corrected image:

In [ ]:
im_pbcor = disp('twhya_cont_auto.pbcor.image/', "Beam Corrected Image")
im_pb = disp('twhya_cont_auto.pb/', "Primary Beam")

show(row(im_pb, im_pbcor, im_cont_auto))

It's often very convenient to work in images before primary beam correction because the noise is the same across the field (e.g., this is a clean data set to search for signal) but it's very important to remember to apply this correction before calculating fluxes or intensities for science.